In [2]:
# Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', 999)

In [3]:
# Import Dataset
file_path = r'C:\Users\Computer\Desktop\Github\03 Customer Behavior and Segmentation using Kmeans\Data\Online Retail.xlsx'
df = pd.read_excel(file_path)

In [4]:
# Copy Original DF into a new updated DF
updated_df = df.copy()

In [5]:
# Remove Nulls/Nones/NaNs from CustomerID's
updated_df.dropna(subset=["CustomerID"], inplace=True)

In [6]:
# Update Invoice to include only 6 digit values. No cancellations or other Misc. issues
updated_df["InvoiceNo"] = updated_df["InvoiceNo"].astype("str")

mask = (updated_df["InvoiceNo"].str.match("^\\d{6}$") == True)
updated_df = updated_df[mask]

In [7]:
# Update StockCode to include only 5 digit values and 5 digit values following by a Object

updated_df["StockCode"] = updated_df["StockCode"].astype("str")

mask = (
    (updated_df["StockCode"].str.match("^\\d{5}$") == True)
    | (updated_df["StockCode"].str.match("^\\d{5}[a-zA-Z]+$") == True) 
)
updated_df = updated_df[mask]

In [8]:
updated_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,396370.000000,396370,396370.000000,396370.000000
mean,13.046474,2011-07-11 00:04:30.985594624,2.867983,15301.602071
min,1.000000,2010-12-01 08:26:00,0.000000,12346.000000
25%,2.000000,2011-04-07 11:16:00,1.250000,13975.000000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000
75%,12.000000,2011-10-20 14:41:00,3.750000,16803.000000
max,80995.000000,2011-12-09 12:50:00,649.500000,18287.000000
std,180.733597,NaN,4.264566,1709.982593


In [9]:
# Noticed there was a minimum of 0 for UnitPrice, so I checked how many there were
len(updated_df[updated_df["UnitPrice"] == 0])

33

In [10]:
# Look through where UnitPrice 0
# Nothing unusual about Transactions with UnitPrice 0, outside UnitPrice 0
updated_df[updated_df["UnitPrice"] == 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
9302,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,2010-12-05 14:02:00,0.0,12647.0,Germany
33576,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,2010-12-16 14:36:00,0.0,16560.0,United Kingdom
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,2010-12-21 13:45:00,0.0,14911.0,EIRE
47068,540372,22090,PAPER BUNTING RETROSPOT,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
47070,540372,22553,PLASTERS IN TIN SKULLS,24,2011-01-06 16:41:00,0.0,13081.0,United Kingdom
56674,541109,22168,ORGANISER WOOD ANTIQUE WHITE,1,2011-01-13 15:10:00,0.0,15107.0,United Kingdom
86789,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,16,2011-02-10 13:08:00,0.0,17560.0,United Kingdom
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,2011-03-23 10:25:00,0.0,13239.0,United Kingdom
139453,548318,22055,MINI CAKE STAND HANGING STRAWBERY,5,2011-03-30 12:45:00,0.0,13113.0,United Kingdom
145208,548871,22162,HEART GARLAND RUSTIC PADDED,2,2011-04-04 14:42:00,0.0,14410.0,United Kingdom


In [11]:
# Remove the Rows where UnitPrice was 0
updated_df = updated_df[updated_df["UnitPrice"] > 0.0]

In [12]:
updated_df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,396337.000000,396337,396337.000000,396337.000000
mean,13.012547,2011-07-11 00:04:38.980463616,2.868222,15301.689416
min,1.000000,2010-12-01 08:26:00,0.040000,12346.000000
25%,2.000000,2011-04-07 11:16:00,1.250000,13975.000000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000
75%,12.000000,2011-10-20 14:41:00,3.750000,16803.000000
max,80995.000000,2011-12-09 12:50:00,649.500000,18287.000000
std,179.641245,NaN,4.264663,1709.961948


In [13]:
# What percent we dropped from the original Dataset after Data Cleaning
round(1 - len(updated_df)/len(df), 3)

0.269

In [15]:
# Save Dataset to CSV File
output_path = r'C:\Users\Computer\Desktop\Github\03 Customer Behavior and Segmentation using Kmeans\Data\Cleaned Data Set.csv'
updated_df.to_csv(output_path, index=False)